In [1]:
import numpy as np
# fix random seed for reproducibility
np.random.seed(7)

from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

# One hot encoding
enc = OneHotEncoder()
Y = enc.fit_transform(y[:, np.newaxis]).toarray()

# Scale data to have mean 0 and variance 1 
# which is importance for convergence of the neural network
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data set into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y, test_size=0.5, random_state=2)

n_features = X.shape[1]
n_classes = Y.shape[1]

print('Number of Features: %i' %n_features)
print('Number of Classes: %i' %n_classes)

Number of Features: 4
Number of Classes: 3


/home/micaelverissimo/anaconda3/envs/my_devs/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
def create_mlp_model(input_dim, output_dim,
                        n_neurons,
                        hl_act_func = 'relu',
                        ol_act_func = 'softmax',
                        n_layers=1,
                        loss_func = 'mean_squared_error',
                        optimizer = 'adam',
                        metrics = None,
                        name='model'):
    # Create model
    model = Sequential(name=name)
    for ilayer in range(n_layers):
        model.add(Dense(n_neurons if type(n_neurons) != type([]) else n_neurons[ilayer],
                        input_dim=input_dim, activation=hl_act_func))
    model.add(Dense(output_dim, activation=ol_act_func))

    # Compile model
    model.compile(loss=loss_func, 
                  optimizer=optimizer, 
                  metrics= metrics if metrics != None else ['accuracy'])
    return model

In [8]:
test_model = create_mlp_model(input_dim=n_features,
                                 output_dim=n_classes,
                                 n_neurons=5,
                                 n_layers=1,
                                 name='test_model')

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
test_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 25        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 18        
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________


In [10]:
history_dict = {}

history_callback = test_model.fit(X_train, Y_train,
                             batch_size=5,
                             epochs=50,
                             verbose=0,
                             validation_data=(X_test, Y_test))

score = test_model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

history_dict[test_model.name] = [history_callback, test_model]

Instructions for updating:
Use tf.cast instead.
Test loss: 0.05675400579969088
Test accuracy: 0.9333333333333333
